In [ ]:
import networkx as nx
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from scipy.sparse import coo_matrix
import evaluate

In [ ]:
post = nx.Graph()

In [ ]:
import pickle
with open('/workspace/persistent/ndf.pkl', 'rb') as file:
    # Load the contents of the PKL file
    ndf = pickle.load(file)

In [ ]:
import pickle
test_indices=[]
with open('/workspace/persistent/test_split_kim.pkl', 'rb') as file:
    # Load the contents of the PKL file
    test_split = pickle.load(file)
test_split=test_split.dropna(subset=['edge_media_to_caption'])
test_indices=test_split.index.tolist()
test_split['id']=0
test_split['id']=range(len(test_split))
test_split = test_split.set_index(test_split['id'])
test_target=[]
for i,rows in test_split.iterrows():
    test_target.append(int(test_split['sponsored'][i]))

In [ ]:
import pickle
train_indices=[]
with open('/workspace/persistent/train_split_kim.pkl', 'rb') as file:
    # Load the contents of the PKL file
    train_split = pickle.load(file)
train_split=train_split.dropna(subset=['edge_media_to_caption'])
train_indices=train_split.index.tolist()
train_split['id']=0
train_split['id']=range(len(train_split))
train_split = train_split.set_index(train_split['id'])
train_target=[]
for i,rows in train_split.iterrows():
    train_target.append(int(train_split['sponsored'][i]))

In [ ]:
y_train = [float(train_split['sponsored'][i]) for i in range(len(train_split))] + [float(0)] * (len(post)-len(train_split))

In [ ]:
import pickle
val_indices=[]
with open('/workspace/persistent/eval_split_kim.pkl', 'rb') as file:
    # Load the contents of the PKL file
    eval_split = pickle.load(file)
eval_split=eval_split.dropna(subset=['edge_media_to_caption'])
val_indices=eval_split.index.tolist()
eval_split['id']=0
eval_split['id']=range(len(eval_split))
eval_split = eval_split.set_index(eval_split['id'])
eval_target=[]
for i,rows in eval_split.iterrows():
    eval_target.append(int(eval_split['sponsored'][i]))

In [ ]:
# Transforming the 'sponsored' column into numerical values
ndf['sponsored'] = np.where(ndf['sponsored'], 1, 0)

In [ ]:
category_mapping = {
    'Creators & Celebrities': 1,
    'Publishers': 2,
    'Personal Goods & General Merchandise Stores': 3,
    'General Interest': 4,
    'Non-Profits & Religious Organizations': 5,
    'Transportation & Accomodation Services': 6,
    'Home Services': 7,
    'Business & Utility Services': 8,
    'Home Goods Stores': 9,
    'Lifestyle Services': 10,
    'Local Events': 11,
    'Food & Personal Goods': 12,
    'Professional Services': 13,
    'Content & Apps': 14,
    'Grocery & Convenience Stores': 15,
    'Restaurants': 16,
    'Auto Dealers': 17,
    'Government Agencies': 18,
    'Entities': 19,
    'Geography': 20,
    'Home & Auto': 21
}

ndf['Category'] = ndf['Category'].map(category_mapping)

In [ ]:
count = 0
while count<len(ndf):
    post.add_nodes_from([(str(ndf['shortcode'][count]),{'x':(ndf['edge_media_preview_like'][count]), 'comments': (ndf['edge_media_to_comment'][count]),'followers': 0,'followees':0,'posts':0, 'hashtags':(ndf['hashtag_count'][count]),'tags':float(ndf['tag_count'][count]), 'category':0, 'pics': float(ndf['pic_count'][count]), 'time': float(ndf['taken_at_timestamp'][count]), 'y':ndf['sponsored'][count]})])
    count=count+1

In [ ]:
influencer = nx.Graph()

In [ ]:
influencerdf = pd.read_csv('/workspace/persistent/influencerdf.csv')
influencerdf=influencerdf.dropna(subset=['Category'])


In [ ]:
influencerdf['id']=0
influencerdf['id']=range(len(influencerdf))
influencerdf = influencerdf.set_index(influencerdf['id'])

In [ ]:
category_mapping = {
    'Creators & Celebrities': 1,
    'Publishers': 2,
    'Personal Goods & General Merchandise Stores': 3,
    'General Interest': 4,
    'Non-Profits & Religious Organizations': 5,
    'Transportation & Accomodation Services': 6,
    'Home Services': 7,
    'Business & Utility Services': 8,
    'Home Goods Stores': 9,
    'Lifestyle Services': 10,
    'Local Events': 11,
    'Food & Personal Goods': 12,
    'Professional Services': 13,
    'Content & Apps': 14,
    'Grocery & Convenience Stores': 15,
    'Restaurants': 16,
    'Auto Dealers': 17,
    'Government Agencies': 18,
    'Entities': 19,
    'Geography': 20,
    'Home & Auto': 21
}

influencerdf['Category'] = influencerdf['Category'].map(category_mapping)

In [ ]:
b=influencerdf.merge(ndf,left_on='Name',right_on='Name',how='inner')
b=b.merge(ndf,left_on='shortcode',right_on='shortcode',how='inner')
b['Combined'] = b['Name_x'].str.cat(b['shortcode'], sep='')

In [ ]:
count = 0
while count<len(b):
    post.add_nodes_from([(str(b['Combined'][count]),{'x':0.0, 'comments': 0,'followers':float(b['Followers_x'][count]),'followees':float(b['Followees_x'][count]),'posts':float(b['Posts_x'][count]), 'hashtags':0, 'tags':0,'category':float(b['Category_x'][count]), 'pics': 0, 'time': 0, 'y':float(b['sponsored_y'][count])})])
    count=count+1

In [ ]:
brand = nx.Graph()

In [ ]:
branddf = pd.read_csv('/workspace/persistent/brandsdf.csv')
branddf=branddf.dropna(subset=['Category'])
branddf['tag']=branddf['tag'].astype(str)

In [ ]:
branddf['id']=0
branddf['id']=range(len(branddf))
branddf = branddf.set_index(branddf['id'])
branddf['tag'] = branddf['tag'].str[1:]

In [ ]:
branddf['Category'] = branddf['Category'].map(category_mapping)

In [ ]:
pic = []
shortcode = []

for i, rows in ndf.iterrows():
    row_text = re.sub(r"['\[\]]", '', ndf['brands'][i])  # Remove single quotes, square brackets
    row_matches = re.findall(r'[^,]+', row_text)  # Parse after every comma
    pic.extend(row_matches)
    shortcode.extend([ndf['shortcode'][i]] * len(row_matches))

# Create a DataFrame with the matches and associated values
imgdf = pd.DataFrame({'brand': pic, 'shortcode': shortcode})


In [ ]:
filtered_imgdf = imgdf[imgdf['brand'].isin(branddf['tag'])]
merged_df = pd.merge(filtered_imgdf, branddf, left_on='brand', right_on='tag', how='inner')
branddf=merged_df

In [ ]:
a=imgdf.merge(branddf,left_on='shortcode',right_on='shortcode',how='inner')
a=a.merge(ndf,left_on='shortcode',right_on='shortcode',how='inner')
a['Combined'] = a['brand_x'].str.cat(a['shortcode'], sep='')

In [ ]:
count = 0
while count<len(a):
    post.add_nodes_from([(str(a['Combined'][count]),{'x':0.0, 'comments': 0,'followers':float(a['Followers_x'][count]),'followees':float(a['Followees_x'][count]),'posts':float(a['Posts_x'][count]), 'hashtags':0, 'tags':0, 'category':float(a['Category_x'][count]), 'pics': 0, 'time': 0, 'y':float(a['sponsored'][count])})])
    count=count+1

In [ ]:
# Iterate over each row in ndf
for i, row in a.iterrows():
     post.add_edge(a['Combined'][i], a['shortcode'][i])
for i, row in b.iterrows():
     post.add_edge(b['Combined'][i], b['shortcode'][i])

In [ ]:
post=nx.convert_node_labels_to_integers(post, first_label=0)

In [ ]:
node

In [ ]:
nodes_to_drop = [2934209]
post.remove_nodes_from(nodes_to_drop)

In [ ]:
import numpy as np

def create_mask(indices, size):
    mask = np.zeros(size, dtype=bool)
    mask[indices] = True
    return mask

# Example usage
test_indices = train_indices
train_indices = train_indices
val_indices = val_indices
size = len(post.nodes())

test_mask = create_mask(test_indices, size)
train_mask = create_mask(train_indices, size)
val_mask = create_mask(val_indices, size)

In [ ]:
# Create an empty feature matrix
feature_matrix = np.zeros((len(post.nodes), 10))

# Iterate over the nodes in the 'post' network
for i, node in enumerate(list(post.nodes)):
    # Get the 'like' and 'comments' attributes of each node
    followers = int(post.nodes[node]['followers'])
    comments = int(post.nodes[node]['comments'])
    like = int(post.nodes[node]['x'])
    tipe = int(post.nodes[node]['followees'])
    country = int(post.nodes[node]['posts'])
    size = int(post.nodes[node]['hashtags'])
    language = int(post.nodes[node]['tags'])
    date = int(post.nodes[node]['category'])
    pics = int(post.nodes[node]['pics'])
    time = int(post.nodes[node]['time'])
    tim = int(post.nodes[node]['y'])
    # Assign the feature values to the feature matrix
    feature_matrix[i] = [int(like), comments,followers,tipe,country,size,language,date,pics,time]

# Print the feature matrix
fm=feature_matrix


In [ ]:
import torch
from torch_geometric.data import Data
import networkx as nx
# Extract node features and convert them to a tensor
node_features = torch.tensor(fm, dtype=torch.float)

edge_indices=list(post.edges())
edge_indices = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()

# Optionally, extract edge features if available and convert them to a tensor
edge_features = None  # Replace None with your edge feature extraction logic if needed

truth=[]
truth = [ndf['sponsored'][i] for i in range(len(ndf))] + [int(0)] * (len(post)-len(ndf))
truth=torch.tensor(truth)
test_truth =[test_split['sponsored'][i] for i in range(len(test_split))]
# Create a PyTorch Geometric Data object
data = Data(x=node_features, edge_index=edge_indices, edge_attr=edge_features,y=truth,train_mask=train_mask,test_mask=test_mask,val_mask=val_mask)

In [ ]:
data.x[0]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        return F.log_softmax(x, dim=1)

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset (e.g., Cora)


# Move the data to the device
data = data.to(device)

# Create DataLoader for batching
loader = DataLoader([data], batch_size=1, shuffle=True)

# Instantiate the model and move it to the device
model = GCN(input_dim=10, hidden_dim=16, num_classes=2).to(device)

# Set up the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

# Training loop
model.train()
for epoch in range(200):
    for batch in loader:
        optimizer.zero_grad()
        batch = batch.to(device)
        out = model(data.x, data.edge_index)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

# Evaluate the trained model
model.eval()
pred = model(data.x.to(device), data.edge_index.to(device)).argmax(dim=1).cpu().numpy()

# Convert ground truth labels to numpy array
ground_truth = data.y.cpu().numpy()

# Calculate precision, recall, and F1 score
precision = precision_score(ground_truth, pred)
recall = recall_score(ground_truth, pred)
f1 = f1_score(ground_truth, pred)
accuracy = (pred == ground_truth).mean()

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [ ]:
torch.save(model,'/workspace/persistent/graph_kim')
torch.load('/workspace/persistent/graph_kim')

In [ ]:
# Select the rows corresponding to train_indices using tensor indexing
last_hidden_state = model(data.x, data.edge_index)
selected_rows = last_hidden_state[test_indices]
graph_state=[]
for i in range(len(selected_rows)):
    graph_state.append(torch.tensor(selected_rows[i]))

In [ ]:
import pickle
with open('/workspace/persistent/graph_state_kim_train.pkl', 'wb') as file:
    pickle.dump(graph_state, file)

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

# Define the self-attention module
class SelfAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super(SelfAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)

    def forward(self, x):
        # Apply self-attention
        output, _ = self.attention(x, x, x)

        return output.squeeze()

# Define the parameters
embedding_dim = 2
num_heads = 2

# Create an instance of the self-attention module
self_attention = SelfAttention(embedding_dim, num_heads).cuda()

# Convert the given tensor to a PyTorch tensor and move to GPU

# Apply self-attention to all elements in the tensor
graph_attention = []
with tqdm(total=len(test_split), desc="Applying Attention") as progress_bar:
    for i,rows in test_split.iterrows():
        # Apply self-attention
        output = (graph_state[i].unsqueeze(0).to('cuda'))
        with torch.no_grad():
            output = output.cpu()
        graph_attention.append(output)
        progress_bar.update(1)

In [ ]:
import pickle
with open('/workspace/persistent/graph_attention_kim_train.pkl', 'wb') as file:
    pickle.dump(graph_attention, file)

In [ ]:
graph_representation = []
with tqdm(total=len(test_split), desc="Calculating the Image Score") as progress_bar:
    for i,rows in test_split.iterrows():
        xi=graph_state[i].unsqueeze(0).to('cuda')*graph_attention[i].unsqueeze(0).to('cuda')
        with torch.no_grad():
            xi = xi.cpu()
        graph_representation.append(xi)
        progress_bar.update(1)

In [ ]:
import pickle
with open('/workspace/persistent/graph_representation_kim_train.pkl', 'wb') as file:
    pickle.dump(graph_representation, file)